# Importing core packages

In [12]:
from yahooquery import Ticker
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import xlsxwriter
from scipy import stats
import math
from scipy.stats import percentileofscore as score
from statistics import mean

## Scraping IBOVESPA's index
In this step we scrape IBOVESPA's website to find which stock composes the index and add '.SA' after it so we can use YahooFinance's API.

In [13]:
html = requests.get("http://bvmf.bmfbovespa.com.br/indices/ResumoCarteiraQuadrimestre.aspx?Indice=IBOV&idioma=pt-br").content
soup = BeautifulSoup(html, 'html.parser')
stocks = []

for stock in soup.find_all("span", class_="label", id=lambda x: x and x.endswith('_lblCodigo')):
    if len(stock.text) <= 5:
        stocks.append(stock.text + '.SA')

## Loading the main data we need from the API into a Pandas table
In this step we request data from the API and chose the ones we use in Value Investment strategy.

In [14]:
types = [
    'PsRatio', 'PbRatio',
    'EnterprisesValueEBITDARatio', 'EnterprisesValueRevenueRatio',
    'PeRatio'
]
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for stock in stocks:
    try:
        stock_data = Ticker(stock).get_financial_data(types, trailing=False)
        stock_price = Ticker(stock).history().iloc[-1]['close']
    except AttributeError:
        continue

    try:
        latest_price = stock_price
        pe_ratio = stock_data.iloc[-1]['PeRatio']
        pb_ratio = stock_data.iloc[-1]['PbRatio']
        ps_ratio = stock_data.iloc[-1]['PsRatio']
        ev_to_ebitda = stock_data.iloc[-1]['EnterprisesValueEBITDARatio']
        ev_to_gp = stock_data.iloc[-1]['EnterprisesValueRevenueRatio']
    except:
        continue

    rv_dataframe = rv_dataframe.append(
        pd.Series(
            [
                stock,
                latest_price,
                'N/A',
                pe_ratio,
                'N/A',
                pb_ratio,
                'N/A',
                ps_ratio,
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gp,
                'N/A',
                'N/A'
            ],
            index = rv_columns,
        ),
        ignore_index = True
    )

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABEV3.SA,15.500000,N/A,26.755517,N/A,4.466470,N/A,5.614345,N/A,14.541625,N/A,5.375262,N/A,N/A
1,AZUL4.SA,37.700001,N/A,17.449102,N/A,NaN,N/A,1.851245,N/A,14.178783,N/A,2.824953,N/A,N/A
2,B3SA3.SA,59.990002,N/A,34.469315,N/A,3.466667,N/A,15.660500,N/A,18.022728,N/A,14.292003,N/A,N/A
3,BEEF3.SA,10.370000,N/A,10.161601,N/A,NaN,N/A,0.369996,N/A,11.408404,N/A,0.721609,N/A,N/A
4,BRAP4.SA,64.709999,N/A,7.261883,N/A,1.305564,N/A,6.908135,N/A,NaN,N/A,NaN,N/A,N/A
5,BRDT3.SA,22.100000,N/A,9.396875,N/A,3.648360,N/A,0.364699,N/A,8.874180,N/A,0.393142,N/A,N/A
6,BRFS3.SA,22.389999,N/A,33.632715,N/A,3.996400,N/A,0.880331,N/A,8.004146,N/A,1.341536,N/A,N/A
7,BRKM5.SA,23.240000,N/A,3448.873484,N/A,3.270056,N/A,0.435672,N/A,44.334268,N/A,1.151380,N/A,N/A
8,BRML3.SA,10.010000,N/A,9.720809,N/A,1.432519,N/A,12.143966,N/A,8.347140,N/A,13.698159,N/A,N/A
9,CCRO3.SA,13.440000,N/A,51.907562,N/A,4.385238,N/A,3.763017,N/A,9.617311,N/A,4.906366,N/A,N/A


## Dealing with EMPTY data
When using a public API we probably will find some missing data and we have to deal with it, in this step we can either remove stocks that miss data or use the mean of everyother stocks to fill the values. I chose to remove them.

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
1,AZUL4.SA,37.700001,N/A,17.449102,N/A,NaN,N/A,1.851245,N/A,14.178783,N/A,2.824953,N/A,N/A
3,BEEF3.SA,10.370000,N/A,10.161601,N/A,NaN,N/A,0.369996,N/A,11.408404,N/A,0.721609,N/A,N/A
4,BRAP4.SA,64.709999,N/A,7.261883,N/A,1.305564,N/A,6.908135,N/A,NaN,N/A,NaN,N/A,N/A
19,ECOR3.SA,13.150000,N/A,NaN,N/A,24.148116,N/A,2.453314,N/A,13.284246,N/A,3.996237,N/A,N/A
23,EMBR3.SA,8.560000,N/A,NaN,N/A,0.963757,N/A,0.703058,N/A,173.151033,N/A,8.200882,N/A,N/A
44,PRIO3.SA,66.129997,N/A,NaN,N/A,3.606865,N/A,5.151256,N/A,5.028273,N/A,3.722066,N/A,N/A
50,SUZB3.SA,56.369999,N/A,NaN,N/A,3.164190,N/A,2.298152,N/A,30.649095,N/A,4.362435,N/A,N/A
57,VVAR3.SA,16.129999,N/A,NaN,N/A,9.846522,N/A,0.565884,N/A,-57.042356,N/A,0.880482,N/A,N/A


In [16]:
#for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    #rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe.dropna(inplace=True)
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


#  Calculating Value Percentiles
Now we have to calculate Value Percentile for every stock in the table.

In [17]:
metrics = {
'Price-to-Earnings Ratio' : 'PE Percentile',
'Price-to-Book Ratio' : 'PB Percentile',
'Price-to-Sales Ratio' : 'PS Percentile',
'EV/EBITDA' : 'EV/EBITDA Percentile',
'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABEV3.SA,15.500000,N/A,26.755517,0.576923,4.466470,0.711538,5.614345,0.807692,14.541625,0.596154,5.375262,0.75,N/A
2,B3SA3.SA,59.990002,N/A,34.469315,0.673077,3.466667,0.557692,15.660500,0.980769,18.022728,0.730769,14.292003,0.961538,N/A
5,BRDT3.SA,22.100000,N/A,9.396875,0.0961538,3.648360,0.634615,0.364699,0.0769231,8.874180,0.346154,0.393142,0.0384615,N/A
6,BRFS3.SA,22.389999,N/A,33.632715,0.634615,3.996400,0.653846,0.880331,0.230769,8.004146,0.288462,1.341536,0.230769,N/A
7,BRKM5.SA,23.240000,N/A,3448.873484,1,3.270056,0.538462,0.435672,0.0961538,44.334268,0.980769,1.151380,0.173077,N/A
8,BRML3.SA,10.010000,N/A,9.720809,0.115385,1.432519,0.211538,12.143966,0.923077,8.347140,0.307692,13.698159,0.942308,N/A
9,CCRO3.SA,13.440000,N/A,51.907562,0.865385,4.385238,0.692308,3.763017,0.653846,9.617311,0.403846,4.906366,0.653846,N/A
10,CIEL3.SA,3.530000,N/A,9.898648,0.153846,2.438298,0.403846,1.997504,0.423077,7.340605,0.192308,2.781599,0.403846,N/A
11,CMIG4.SA,14.180000,N/A,11.707284,0.211538,1.307783,0.134615,0.865436,0.211538,4.890791,0.0769231,1.289058,0.211538,N/A
12,COGN3.SA,4.600000,N/A,36.278459,0.711538,1.170990,0.0961538,2.662262,0.557692,12.684343,0.519231,4.207035,0.596154,N/A


## Calculating the RV Score

In [18]:
for row in rv_dataframe.index:
    values_percentiles = []
    for metric in metrics.keys():
        values_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(values_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,ABEV3.SA,15.500000,N/A,26.755517,0.576923,4.466470,0.711538,5.614345,0.807692,14.541625,0.596154,5.375262,0.75,0.688462
2,B3SA3.SA,59.990002,N/A,34.469315,0.673077,3.466667,0.557692,15.660500,0.980769,18.022728,0.730769,14.292003,0.961538,0.780769
5,BRDT3.SA,22.100000,N/A,9.396875,0.0961538,3.648360,0.634615,0.364699,0.0769231,8.874180,0.346154,0.393142,0.0384615,0.238462
6,BRFS3.SA,22.389999,N/A,33.632715,0.634615,3.996400,0.653846,0.880331,0.230769,8.004146,0.288462,1.341536,0.230769,0.407692
7,BRKM5.SA,23.240000,N/A,3448.873484,1,3.270056,0.538462,0.435672,0.0961538,44.334268,0.980769,1.151380,0.173077,0.557692
8,BRML3.SA,10.010000,N/A,9.720809,0.115385,1.432519,0.211538,12.143966,0.923077,8.347140,0.307692,13.698159,0.942308,0.5
9,CCRO3.SA,13.440000,N/A,51.907562,0.865385,4.385238,0.692308,3.763017,0.653846,9.617311,0.403846,4.906366,0.653846,0.653846
10,CIEL3.SA,3.530000,N/A,9.898648,0.153846,2.438298,0.403846,1.997504,0.423077,7.340605,0.192308,2.781599,0.403846,0.315385
11,CMIG4.SA,14.180000,N/A,11.707284,0.211538,1.307783,0.134615,0.865436,0.211538,4.890791,0.0769231,1.289058,0.211538,0.169231
12,COGN3.SA,4.600000,N/A,36.278459,0.711538,1.170990,0.0961538,2.662262,0.557692,12.684343,0.519231,4.207035,0.596154,0.496154


## Selecting the 7 Best Value Stocks
In this step we sort the stocks descendly by RV Score and remove the rest.

In [9]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:15]
rv_dataframe.reset_index(inplace=True, drop=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GOAU4.SA,11.070000,N/A,18.560000,0.403846,0.988974,0.0769231,0.246874,0.0192308,4.800964,0.0576923,0.566844,0.0769231,0.126923
1,CMIG4.SA,14.180000,N/A,11.707284,0.211538,1.307783,0.134615,0.865436,0.211538,4.890791,0.0769231,1.289058,0.211538,0.169231
2,ENBR3.SA,19.540001,N/A,9.860173,0.134615,1.440202,0.230769,0.973058,0.25,5.165203,0.0961538,1.262669,0.192308,0.180769
3,PCAR3.SA,72.709999,N/A,24.698082,0.519231,2.191029,0.326923,0.447942,0.115385,4.457097,0.0384615,0.291578,0.0192308,0.203846
4,JBSS3.SA,23.780001,N/A,16.433121,0.326923,2.239406,0.384615,0.348379,0.0576923,6.468315,0.153846,0.580120,0.0961538,0.203846
5,ELET3.SA,36.040001,N/A,3.920963,0.0192308,0.810940,0.0192308,1.913841,0.384615,7.443956,0.221154,3.158648,0.490385,0.226923
6,CSNA3.SA,30.860001,N/A,7.928169,0.0576923,2.203276,0.346154,0.782063,0.134615,9.392858,0.365385,1.832550,0.269231,0.234615
7,BRDT3.SA,22.100000,N/A,9.396875,0.0961538,3.648360,0.634615,0.364699,0.0769231,8.874180,0.346154,0.393142,0.0384615,0.238462
8,ELET6.SA,36.130001,N/A,3.966604,0.0384615,0.820379,0.0384615,1.936119,0.403846,7.443956,0.221154,3.158648,0.490385,0.238462
9,TIMP3.SA,14.510000,N/A,10.426259,0.173077,1.719050,0.269231,2.199075,0.480769,3.762608,0.0192308,2.307351,0.346154,0.257692


## Calculating the Number of Shares to Buy
To calculate the number of shares to buy we simply divide our portfolio in equal sizes of each stock. In the future I will apply more sofisticated methods into it.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = 10000
    #float(input('Insert your portfolio size: '))
    
portfolio_input()
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])

rv_dataframe

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GOAU4.SA,11.070000,60,18.560000,0.403846,0.988974,0.0769231,0.246874,0.0192308,4.800964,0.0576923,0.566844,0.0769231,0.126923
1,CMIG4.SA,14.180000,47,11.707284,0.211538,1.307783,0.134615,0.865436,0.211538,4.890791,0.0769231,1.289058,0.211538,0.169231
2,ENBR3.SA,19.540001,34,9.860173,0.134615,1.440202,0.230769,0.973058,0.25,5.165203,0.0961538,1.262669,0.192308,0.180769
3,PCAR3.SA,72.709999,9,24.698082,0.519231,2.191029,0.326923,0.447942,0.115385,4.457097,0.0384615,0.291578,0.0192308,0.203846
4,JBSS3.SA,23.780001,28,16.433121,0.326923,2.239406,0.384615,0.348379,0.0576923,6.468315,0.153846,0.580120,0.0961538,0.203846
5,ELET3.SA,36.040001,18,3.920963,0.0192308,0.810940,0.0192308,1.913841,0.384615,7.443956,0.221154,3.158648,0.490385,0.226923
6,CSNA3.SA,30.860001,21,7.928169,0.0576923,2.203276,0.346154,0.782063,0.134615,9.392858,0.365385,1.832550,0.269231,0.234615
7,BRDT3.SA,22.100000,30,9.396875,0.0961538,3.648360,0.634615,0.364699,0.0769231,8.874180,0.346154,0.393142,0.0384615,0.238462
8,ELET6.SA,36.130001,18,3.966604,0.0384615,0.820379,0.0384615,1.936119,0.403846,7.443956,0.221154,3.158648,0.490385,0.238462
9,TIMP3.SA,14.510000,45,10.426259,0.173077,1.719050,0.269231,2.199075,0.480769,3.762608,0.0192308,2.307351,0.346154,0.257692


## Exporting the Pandas table to Excel
We take advantage of xlsxwriter library to export our table into Excel's table.

In [11]:
writer = pd.ExcelWriter('Value_Strategy_IBOVESPA.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy - IBOVESPA', index=False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'R$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
    {
        'num_format':'0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy - IBOVESPA'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy - IBOVESPA'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    
writer.save()